# Kaggle Dogs VS Cats

## Basic Lenet-5

In [1]:
import random
import os

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
from tensorflow.keras import layers, callbacks, metrics, optimizers, models
import keras_tuner as kt
import numpy as np
import skimage.io as io
import data_sequence

2022-04-05 23:35:02.535419: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 23:35:02.546979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 23:35:02.547517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
class DogCatSequence(tf.keras.utils.Sequence):
    
    def __init__(self, files_folder, files_list, batch_size):
        if batch_size > len(files_list):
            raise ValueError('Batch size is bigger than length of files list')
            
        self._files_list = files_list
        self._batch_size = batch_size
        self._files_folder = files_folder

    def __getitem__(self, index):
        end = min((index + 1) * self._batch_size, len(self._files_list) - 1)
        batch_files_list = self._files_list[index * self._batch_size: end]

        batch_x = np.array([io.imread(os.path.join(self._files_folder, filename)) for filename in batch_files_list])
        batch_y = np.array([float(filename.startswith('dog')) for filename in batch_files_list])

        return batch_x, batch_y

    def __len__(self):
        return int(np.ceil(len(self._files_list) / self._batch_size))


def f_score(ytrue, ypred, threshold=0.5, epsilon=10e-7):
    # casting ytrue and ypred as float dtype
    ytrue = tf.cast(ytrue, tf.float32)
    ypred = tf.cast(ypred, tf.float32)

    # setting values of ypred greater than the set threshold to 1 while those lesser to 0
    ypred = tf.cast(tf.greater_equal(ypred, tf.constant(threshold)), tf.float32)

    tp = tf.reduce_sum(ytrue*ypred) # calculating true positives
    predicted_positive = tf.reduce_sum(ypred) # calculating predicted positives
    actual_positive = tf.reduce_sum(ytrue) # calculating actual positives
    
    precision = tp/(predicted_positive+epsilon) # calculating precision
    recall = tp/(actual_positive+epsilon) # calculating recall
    
    # calculating fbeta
    fb = 2 * precision*recall / (precision + recall + epsilon)

    return fb


def print_errors(model, train_seq, test_seq):
    train_f_score = model.evaluate(train_seq)[1]
    test_f_score = model.evaluate(test_seq)[1]

    print('train error = ', 1 - train_f_score)
    print('test error = ', 1 - test_f_score)
    
    
def train_model(model):
    model_check_point = callbacks.ModelCheckpoint(filepath='dogs_vs_cats_lenet5_weights_{epoch}.h5',
                                              save_weights_only=True,
                                              save_best_only=True,
                                              monitor='val_loss',
                                              mode='min',
                                              verbose=True)

    early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                            mode='min',
                                            min_delta=10e-3,
                                            verbose=True,
                                            patience=10,
                                            restore_best_weights=True)


    train_epochs = 100

    train_callbacks = [model_check_point, early_stopping]

    model.fit(train_seq, epochs=train_epochs, validation_data=val_seq, callbacks=train_callbacks)
    
    return model

def tune_model(model_func, train_sequence, validation_sequence):
    hp = kt.HyperParameters()

    tuner = kt.Hyperband(
        model_func,
        objective="val_loss",
        max_epochs=15,
        directory='tuner',
        project_name=str(model_func),
    )

    train_epochs = 10
    tuner.search(train_sequence, epochs=train_epochs, validation_data=validation_sequence)
    
    return tuner

def get_dataset_sequences(images_folder, batch_size):
    images_list = sorted(os.listdir(images_folder))
    random.seed(42)
    random.shuffle(images_list)

    train_part = 0.9
    val_part = 0.05
    test_part = 0.05

    get_last_index = lambda part: int(part * len(images_list))

    train_images_list = images_list[: get_last_index(train_part)]
    val_images_list = images_list[get_last_index(train_part): get_last_index(train_part + val_part)]
    test_images_list = images_list[get_last_index(train_part + val_part):
                                       get_last_index(train_part + val_part + test_part)]
    train_seq = DogCatSequence(images_folder, train_images_list, batch_size)
    val_seq = DogCatSequence(images_folder, val_images_list, batch_size)
    test_seq = DogCatSequence(images_folder, test_images_list, batch_size)
    
    return train_seq, val_seq, test_seq

In [55]:
train_seq, val_seq, test_seq = get_dataset_sequences('train-64')

## Basic Lenet-5

In [10]:
def lenet5_v1(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    nn = layers.Rescaling(1.0 / 255.0)(inputs)
    nn = layers.Conv2D(filters=6, kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    nn = layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    nn = layers.Flatten()(nn)
    nn = layers.Dense(400, activation='relu')(nn)
    nn = layers.Dense(120, activation='relu')(nn)
    nn = layers.Dense(84, activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)
    return models.Model(inputs, outputs)

In [11]:
model = lenet5_v1(train_seq[0][0].shape[1:])

train_metrics = [f_score,
                 metrics.Precision(thresholds=0.5),
                 metrics.Recall(thresholds=0.5)]

train_optimizer = optimizers.Adam(learning_rate=0.001)

train_loss = 'binary_crossentropy'

model.compile(loss=train_loss,
              optimizer=train_optimizer,
              metrics=train_metrics)

In [12]:
train_model(model)

Epoch 1/100
351/352 [============================>.] - ETA: 0s - loss: 1.4011 - f_score: 0.5428 - precision_1: 0.5430 - recall_1: 0.6137
Epoch 1: val_loss improved from inf to 0.68042, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 11s 29ms/step - loss: 1.3990 - f_score: 0.5430 - precision_1: 0.5430 - recall_1: 0.6139 - val_loss: 0.6804 - val_f_score: 0.5652 - val_precision_1: 0.5822 - val_recall_1: 0.5584
Epoch 2/100
351/352 [============================>.] - ETA: 0s - loss: 0.6473 - f_score: 0.6199 - precision_1: 0.6176 - recall_1: 0.6442
Epoch 2: val_loss improved from 0.68042 to 0.65341, saving model to dogs_vs_cats_lenet5_weights_2.h5
352/352 [==============================] - 10s 28ms/step - loss: 0.6474 - f_score: 0.6202 - precision_1: 0.6180 - recall_1: 0.6447 - val_loss: 0.6534 - val_f_score: 0.6624 - val_precision_1: 0.5955 - val_recall_1: 0.7476
Epoch 3/100
352/352 [==============================] - ETA: 0s - loss: 0.6033 - f_scor

In [13]:
print_errors(model, train_seq, test_seq)

20/20 [==============================] - 1s 27ms/step - loss: 0.6495 - f_score: 0.6705 - precision_1: 0.6068 - recall_1: 0.7595
train error =  0.2862144112586975
test error =  0.329542338848114


## Basic Lenet-5 with tuning

In [14]:
def lenet5_v2(hp):
    
    inputs = layers.Input(shape=(64, 64, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=10, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=10, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=50, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=60, max_value=100, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [21]:
tuner = tune_model(lenet5_v2)

Trial 30 Complete [00h 02m 32s]
val_loss: 0.5534921884536743

Best val_loss So Far: 0.515401303768158
Total elapsed time: 00h 22m 10s
INFO:tensorflow:Oracle triggered exit


In [22]:
lenet5_v2_best_model = tuner.get_best_models()[0]
lenet5_v2_best_model = train_model(lenet5_v2_best_model)

Epoch 1/100
351/352 [============================>.] - ETA: 0s - loss: 0.4684 - f_score: 0.7750 - precision: 0.7768 - recall: 0.7819
Epoch 1: val_loss improved from inf to 0.50380, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 11s 29ms/step - loss: 0.4685 - f_score: 0.7751 - precision: 0.7766 - recall: 0.7822 - val_loss: 0.5038 - val_f_score: 0.7344 - val_precision: 0.8192 - val_recall: 0.6719
Epoch 2/100
352/352 [==============================] - ETA: 0s - loss: 0.4109 - f_score: 0.8078 - precision: 0.8117 - recall: 0.8113
Epoch 2: val_loss did not improve from 0.50380
352/352 [==============================] - 10s 28ms/step - loss: 0.4109 - f_score: 0.8078 - precision: 0.8117 - recall: 0.8113 - val_loss: 0.5213 - val_f_score: 0.7787 - val_precision: 0.7263 - val_recall: 0.8454
Epoch 3/100
350/352 [============================>.] - ETA: 0s - loss: 0.3421 - f_score: 0.8461 - precision: 0.8458 - recall: 0.8524
Epoch 3: val_loss did not impro

In [23]:
print_errors(lenet5_v2_best_model, train_seq, test_seq)

20/20 [==============================] - 1s 26ms/step - loss: 0.5472 - f_score: 0.6901 - precision: 0.7789 - recall: 0.6297
train error =  0.1928519606590271
test error =  0.3098623752593994


## Lenet-5 with additional conv layer

In [24]:
def lenet5_v3(hp):
    
    inputs = layers.Input(shape=(64, 64, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=10, max_value=30, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=20, max_value=40, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=50, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=60, max_value=100, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [25]:
tuner = tune_model(lenet5_v3)

Trial 30 Complete [00h 02m 34s]
val_loss: 0.5273486375808716

Best val_loss So Far: 0.4456341564655304
Total elapsed time: 00h 29m 04s
INFO:tensorflow:Oracle triggered exit


In [26]:
lenet5_v3_best_model = tuner.get_best_models()[0]
lenet5_v3_best_model = train_model(lenet5_v3_best_model)

Epoch 1/100
351/352 [============================>.] - ETA: 0s - loss: 0.3684 - f_score: 0.8283 - precision: 0.8333 - recall: 0.8299
Epoch 1: val_loss improved from inf to 0.43387, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 11s 30ms/step - loss: 0.3684 - f_score: 0.8284 - precision: 0.8335 - recall: 0.8301 - val_loss: 0.4339 - val_f_score: 0.8081 - val_precision: 0.7725 - val_recall: 0.8517
Epoch 2/100
351/352 [============================>.] - ETA: 0s - loss: 0.3341 - f_score: 0.8490 - precision: 0.8499 - recall: 0.8529
Epoch 2: val_loss did not improve from 0.43387
352/352 [==============================] - 10s 29ms/step - loss: 0.3345 - f_score: 0.8488 - precision: 0.8502 - recall: 0.8523 - val_loss: 0.4832 - val_f_score: 0.7683 - val_precision: 0.8349 - val_recall: 0.7177
Epoch 3/100
351/352 [============================>.] - ETA: 0s - loss: 0.3072 - f_score: 0.8629 - precision: 0.8626 - recall: 0.8686
Epoch 3: val_loss did not impro

In [27]:
print_errors(lenet5_v3_best_model, train_seq, test_seq)

20/20 [==============================] - 1s 27ms/step - loss: 0.4642 - f_score: 0.7864 - precision: 0.7400 - recall: 0.8465
train error =  0.14125597476959229
test error =  0.21361839771270752


## Lenet5 with two additional conv layers and 128x128 images

In [4]:
train_seq, val_seq, test_seq = get_dataset_sequences('train-128')

(64, 128, 128, 3)


In [5]:
def lenet5_v4(hp):
    
    inputs = layers.Input(shape=(128, 128, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=20, max_value=40, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=40, max_value=60, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_4', min_value=60, max_value=80, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=80, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=40, max_value=80, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [6]:
tuner = tune_model(lenet5_v4, train_seq, val_seq)

Trial 30 Complete [00h 07m 11s]
val_loss: 0.38811683654785156

Best val_loss So Far: 0.35724955797195435
Total elapsed time: 01h 09m 22s
INFO:tensorflow:Oracle triggered exit


In [7]:
lenet5_v4_best_model = tuner.get_best_models()[0]
lenet5_v4_best_model = train_model(lenet5_v4_best_model)

Epoch 1/100
352/352 [==============================] - ETA: 0s - loss: 0.2491 - f_score: 0.8883 - precision: 0.8867 - recall: 0.8942
Epoch 1: val_loss improved from inf to 0.33639, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 28s 78ms/step - loss: 0.2491 - f_score: 0.8883 - precision: 0.8867 - recall: 0.8942 - val_loss: 0.3364 - val_f_score: 0.8575 - val_precision: 0.8460 - val_recall: 0.8754
Epoch 2/100
352/352 [==============================] - ETA: 0s - loss: 0.2141 - f_score: 0.9077 - precision: 0.9054 - recall: 0.9130
Epoch 2: val_loss did not improve from 0.33639
352/352 [==============================] - 27s 77ms/step - loss: 0.2141 - f_score: 0.9077 - precision: 0.9054 - recall: 0.9130 - val_loss: 0.4146 - val_f_score: 0.8201 - val_precision: 0.9074 - val_recall: 0.7571
Epoch 3/100
352/352 [==============================] - ETA: 0s - loss: 0.1729 - f_score: 0.9270 - precision: 0.9266 - recall: 0.9304
Epoch 3: val_loss did not impro

In [8]:
print_errors(lenet5_v4_best_model, train_seq, test_seq)

20/20 [==============================] - 1s 34ms/step - loss: 0.3997 - f_score: 0.8284 - precision: 0.8120 - recall: 0.8544
train error =  0.08293211460113525
test error =  0.17155903577804565


## Lenet5 with three additional conv layers and 256x256 images

In [3]:
train_seq, val_seq, test_seq = get_dataset_sequences('train-256', 32)

In [4]:
def lenet5_v5(hp):
    
    inputs = layers.Input(shape=(256, 256, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=20, max_value=40, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=40, max_value=60, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_4', min_value=60, max_value=80, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_5', min_value=80, max_value=100, step=1),
                        kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=80, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=40, max_value=80, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [5]:
tuner = tune_model(lenet5_v5, train_seq, val_seq)

Trial 30 Complete [00h 31m 26s]
val_loss: 0.22479461133480072

Best val_loss So Far: 0.2145802527666092
Total elapsed time: 04h 46m 47s
INFO:tensorflow:Oracle triggered exit


In [6]:
lenet5_v5_best_model = tuner.get_best_models()[0]
lenet5_v5_best_model = train_model(lenet5_v5_best_model)

Epoch 1/100
704/704 [==============================] - ETA: 0s - loss: 0.1142 - f_score: 0.9531 - precision: 0.9490 - recall: 0.9600
Epoch 1: val_loss improved from inf to 0.22811, saving model to dogs_vs_cats_lenet5_weights_1.h5
704/704 [==============================] - 91s 128ms/step - loss: 0.1142 - f_score: 0.9531 - precision: 0.9490 - recall: 0.9600 - val_loss: 0.2281 - val_f_score: 0.9051 - val_precision: 0.9121 - val_recall: 0.9006
Epoch 2/100
704/704 [==============================] - ETA: 0s - loss: 0.0986 - f_score: 0.9600 - precision: 0.9568 - recall: 0.9657
Epoch 2: val_loss did not improve from 0.22811
704/704 [==============================] - 90s 128ms/step - loss: 0.0986 - f_score: 0.9600 - precision: 0.9568 - recall: 0.9657 - val_loss: 0.2978 - val_f_score: 0.8880 - val_precision: 0.9324 - val_recall: 0.8486
Epoch 3/100
704/704 [==============================] - ETA: 0s - loss: 0.0851 - f_score: 0.9668 - precision: 0.9655 - recall: 0.9696
Epoch 3: val_loss did not imp

In [8]:
print_errors(tuner.get_best_models()[0], train_seq, test_seq)

40/40 [==============================] - 2s 48ms/step - loss: 0.2571 - f_score: 0.8742 - precision: 0.8976 - recall: 0.9019
train error =  0.038176119327545166
test error =  0.12582576274871826
